In [ ]:
%%time
#################################
#将384点板剪裁成384格,按板名命名文件夹保存各自的单藻落

import os 
import cv2
import numpy as np
import pylab
import matplotlib.pyplot as plt


def file_name2(file_dir): #识别路径目录下包含的所有png格式照片
    #注意返回至文件夹中所有文件，会打开深层文件夹查找
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.png':
                L.append(os.path.join(root, file))
    return L

def cv_imread(filePath):#解决中文不识别问题
    cv_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)
    return cv_img 

def divide_spot(file_name):
    img = cv_imread(file_name)
    file_name=file_name.split(".png")[0]
    os.mkdir(file_name) 
    #print("图像的形状,返回一个图像的(行数,列数,通道数):",img.shape)
    w=img.shape[1]
    h=img.shape[0]
    h0=h/16
    w0=w/24   
    j=0
    while j<16:
        i=0
        while i<24:
            img_names=file_name+"/"+str(24*j+i+1)+".png"
            a=round(j*h0)#注意，此时仍为四舍五入取法获取区间，仍有一个像素的遗漏或重叠
            b=round((j+1)*h0)
            c=round(i*w0)
            d=round((i+1)*w0)
            img0=img[a:b,c:d]
            cv2.imencode('.png', img0)[1].tofile(img_names)
            i+=1
        j+=1
    


for i in file_name2('/path_to_the_picture'):#注意会遍历文件夹下所有文件，小心存储
    #if "xxx"in i:################注意更换
    divide_spot(i)

In [ ]:
%%time
##############存储符合RGB范围的区域值，保存HSV格式###########
import os 
import cv2
import numpy as np
import pylab
import matplotlib.pyplot as plt

def file_name1(file_dir): #识别路径目录下包含的所有‘Run’字符串的文件夹，注意更改
    #注意返回至文件夹中所有文件，会打开深层文件夹查找
    F=[] 
    for root, dirs, files in os.walk(file_dir):
        for foldername in dirs:
            if "-" in foldername:##########################################注意更改########################
                F.append(os.path.join(root, foldername))
    return F

def file_name2(file_dir): #识别路径目录下包含的所有png格式照片
    #注意返回至文件夹中所有文件，会打开深层文件夹查找
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.png':
                L.append(os.path.join(root, file))
    return L

def cv_imread(filePath):#解决中文不识别问题
    cv_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)
    return cv_img 
 
def convert_csv(file_name,f1): #注意32位图，BGR格式
    img = cv_imread(file_name)
    m=img.shape[0]
    n=img.shape[1]
    m1=0
    while m1<m:
        n1=0
        while n1<n:
            (b,g,r,h) = img[m1,n1]#读取(0,0)像素，Python中图像像素是按B,G,R顺序存储的
            f1.write(file_name.split("thefilename/")[-1]+","+str(m1)+","+str(n1)+","+str(b)+","+str(g)+","+str(r)+","+str(h)+"\n")#注意与单块板存储的区别
            n1+=1
        m1+=1

def convert_hsv(file_name,f1): #注意32位图时，HSV格式
    imgy = cv_imread(file_name)
    img=cv2.cvtColor(imgy,cv2.COLOR_BGR2HSV)#HSV空间
    #gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #img=gray.reshape((-1))

    m=img.shape[0]
    n=img.shape[1]
    m1=0
    while m1<m:
        n1=0
        while n1<n:

            if len(img[m1,n1])==3:
                (b,g,r) = img[m1,n1]#读取(0,0)像素，Python中图像像素是按H,S,V顺序存储的
            elif len(img[m1,n1])==4:
                (b,g,r,h) = img[m1,n1]
   

            if ((b<10 or g<10 or r<10) and (int(b)+int(g)+int(r))<70) or ((b<6 or g<6 or r<6) and (int(b)+int(g)+int(r))<75):
                flag=0
           
            else:
             
            
                f1.write(file_name.split("pam/")[-1].replace(",",";")+","+str(m1)+","+str(n1)+","+str(b)+","+str(g)+","+str(r)+"\n")
              
                
                #f1.write(file_name.split("117\\")[-1]+","+str(m1)+","+str(n1)+","+str(b1)+","+str(g1)+","+str(r1)+"\n")#注意与单块板存储的区别
            n1+=1
        m1+=1

def change_black(image):#将图片中灰色部分转换为黑色，查看识别用
    
    img = cv_imread(image)
    m=img.shape[0]
    n=img.shape[1]
    m1=0
    while m1<m:
        n1=0
        while n1<n:
            (b,g,r,h) = img[m1,n1]#读取(0,0)像素，HSV
            if ((b<10 or g<10 or r<10) and (int(b)+int(g)+int(r))<70) or ((b<6 or g<6 or r<6) and (int(b)+int(g)+int(r))<75):#
                #f.write(str(b)+","+str(g)+","+str(r)+","+str(int(b)+int(g)+int(r))+"\n")
                b=255
                g=255
                r=255
                img[m1,n1]=(b,g,r)
            n1+=1
        m1+=1
    return img          
        
        

f1=open('pamnew_fhsv.csv','w') 
f1.write("name"+",m1,n1,"+"H"+","+"S"+","+"V"+"\n")
#f2=open('pam_frgb.csv','a') 
#f2.write("name"+",m1,n1,"+"B"+","+"G"+","+"R"+","+"H"+"\n")
for j in file_name1('/storage3/pamgreen20230925/pam'):
    print (j)
    
    for i in file_name2(j):
        
        convert_hsv(i,f1)
        
        #查看用################
        #template = change_black(i)
        #new_names=i.split(".png")[0]+"_70_756.png"
        #cv2.imencode('.png', template)[1].tofile(new_names)
        
f1.close()
#f2.close()

In [ ]:
%%time
import numpy as np
from scipy import stats
##########获取藻落点个数与众数###############
f1=open('pamnew_fhsv.csv','r')
f2=open('pamnew_fhsvuniqall.csv','w')


#np.median(nums)  
#np.mean(nums)  
#C:\Users\JK_GKN\Desktop\pam\L3\L3-2\1.png,16,28,64,247,64
dictseq={}
a=f1.readline().strip()
a=f1.readline().strip()
while 1:
    a=f1.readline().strip()
    if a=="":break
    b=a.split(",")
    name=str(b[0])
    h=int(b[3])
    s=int(b[4])
    v=int(b[5])
    if name in dictseq:
        dictseq[name][0].append(h)
        dictseq[name][1].append(s)
        dictseq[name][2].append(v)
    else:
        dictseq[name]=[[h],[s],[v]]
    
print(len(dictseq))
f2.write("name,num1,num2,num3,modeh,modes,modev,meanh,means,meanv,medianh,medians,medianv,allh,alls,allv\n")
for key in dictseq:
    f2.write(str(key)+","+str(len(dictseq[key][0]))+","+str(len(dictseq[key][1]))+","+str(len(dictseq[key][2]))+
             ","+str(stats.mode(dictseq[key][0])[0][0])+","+str(stats.mode(dictseq[key][1])[0][0])+","+str(stats.mode(dictseq[key][2])[0][0])+
             ","+str(np.mean(dictseq[key][0]))+","+str(np.mean(dictseq[key][1]))+","+str(np.mean(dictseq[key][2]))+
             ","+str(np.median(dictseq[key][0]))+","+str(np.median(dictseq[key][1]))+","+str(np.median(dictseq[key][2]))+
             ","+str(np.sum(dictseq[key][0]))+","+str(np.sum(dictseq[key][1]))+","+str(np.sum(dictseq[key][2]))+"\n")

f1.close()
f2.close()

print("The end!")

In [ ]:
%%time
import numpy as np
from scipy import stats
##########计算为单一列###############
f2=open('pamnew_fhsvuniqalluniq.csv','w')
f1=open('pamnew_fhsvuniqall.csv','r')
a=f1.readline().strip()
f2.write("name,value\n")
while 1:
    a=f1.readline().strip()
    if a=="":break
    b=a.split(",")
    c=float(b[1])
    d=float(b[7])
    e=float(b[10])
    f=float(b[12])
    v=161.25*e-2.8816###################根据色度条对应数值获得回归公式
    vv=round(v,3)
    f2.write(b[0]+","+b[1]+","+b[-1]+","+str(vv)+"\n")

f1.close()
f2.close()

print("The end!")

In [ ]:
%%time
import numpy as np
from scipy import stats

f2=open('pamnew_fhsvuniqname.csv','w')
f1=open('pamnew_fhsvuniqalluniq.csv','r')
#################转换数字序列为板的行列序列###########
def changeseq_384(num):
    a=(num-1)//24
    tail=(num-1)%24+1
    head=chr(65+a)
    body=head+str(tail)
    return body
#name,num,modev,meanv,medianv,allv
#f2.write("name,num,modev,meanv,medianv,allv\n")
while 1:
    a=f1.readline().strip()
    if a=="":break
    if "name" in a:
        f2.write(a+"\n")
    else:
        try:
            #412-H Group_20220412 H2 group_D6-Ft/21.png,
            b=int(a.split(".png")[0].split("/")[-1])
            c=a.split("/")[0]+"-"+changeseq_384(b)
            d=a.split(",")
            f2.write(c+","+a+"\n")
        except:
            print(a)
            b=int(a.split(".png")[0].split("/")[-1])
            print("1")
            c=a.split("/")[0]+"-"+changeseq_384(b)
            d=a.split(",")
            print("2")
            print(b)
            print(c)
            print(d[1])
            f2.write(c+","+a+"\n")

f1.close()
f2.close()

print("The end!")

In [ ]:
%%time
############################合并不同组数据#############
######################################################
##################以A1-P24为顺序排列各组数据##########
import numpy as np
from scipy import stats

#f1=open('g_fhsvuniqname.csv','r')
#f3=open('g96_fhsvuniqname.csv','r')
f3=open('pamnew_fhsvuniqname.csv','r')
f2=open('pamnew.csv','w')

def changeseq_384(num):
    a=(num-1)//24
    tail=(num-1)%24+1
    head=chr(65+a)
    body=head+str(tail)
    return body
def dictseq_0(dictseq):
    i=1
    while i<385:
        dictseq[changeseq_384(i)]=changeseq_384(i)+",NA"
        i+=1
    return dictseq

    

dictseq={}
b=f3.readline()
f2.write(b)

while 1:
    a=f3.readline().strip()
    if a=="":break
    b=a.split(",")   
    #412-H Group_20220412 H2 group_D6-Ft-A21,
    name="-".join(b[0].replace(" ","-").split("-")[:-1])
    site=b[0].split("-")[-1]
    if name in dictseq:
        dictseq[name][site]=a
    else:
        dictseq1={}
        dictseq[name]=dictseq_0(dictseq1)
        dictseq[name][site]=a
for key in sorted(dictseq) :
    for keys in dictseq[key]:
        if "NA" in dictseq[key][keys]:
            f2.write(key+","+key+"-"+dictseq[key][keys]+",NA,NA,NA\n")
        else:
            f2.write(key+","+dictseq[key][keys].replace(" ","-")+"\n")


        

#f1.close()
f2.close()
f3.close()
print("The end!")